In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import h3
import folium
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from tqdm import tqdm
from geopy.geocoders import Nominatim
import pickle

In [2]:
h = '89110610a4fffff'
print(h3.h3_to_geo(h))
print(h3.h3_to_geo_boundary(h))
print(h3.geo_to_h3(59.658116, 30.1480510195878, 9))

(59.79511633691718, 30.148051019587808)
((59.79418534415885, 30.145347613504633), (59.79336572152563, 30.14802331140712), (59.79429666329383, 30.1507267911911), (59.79604729413994, 30.15075463446571), (59.79686694210071, 30.14807873051579), (59.79593593388555, 30.145375189345877))
8911061843bffff


In [5]:
h3_test_1 = ((30.311958245114507, 60.178148715221994), (30.314663468848707, 60.17733089470366), (30.317405157887038, 60.17826254489239), (30.317441687573208, 60.18001208215914), (30.314736254022115, 60.1808299286797), (30.311994500608964, 60.17989821192894), (30.311958245114507, 60.178148715221994))
map = folium.Map((h3_test_1[0][1],h3_test_1[0][0]), zoom_start=15)
for pt in h3_test_1:
    marker = folium.Marker([pt[1], pt[0]]) #latitude,longitude
    map.add_child(marker)
map

## Наши данные

In [ ]:
import json

with open('spb.geojson') as user_file:
    file_contents = user_file.read()
file_contents

In [ ]:
json_tele2 = []
with open('spb_1.geojson', 'r') as j:
    contents = json.loads(j.read())
for i in contents['features'][0]['geometry']['coordinates'][0]:
    json_tele2.append(i)
    
json_tele2

In [ ]:
json_tele2_df = pd.DataFrame(json_tele2, columns = ['lon', 'lat']) 
json_tele2_df

In [ ]:
spb_hex = pd.read_excel('spb_hex.xlsx', sheet_name='spb_hex', header=0, na_values=['NA'])
spb_hex

### Add coordinates

In [ ]:
spb_hex['coordinates'] = spb_hex.apply(lambda x:  h3.h3_to_geo(x['h3_9']), axis=1)
spb_hex

### Add coordinates in geo formats

In [ ]:
spb_hex['coordinates_geo'] = spb_hex.apply(lambda x:  gpd.points_from_xy(spb_hex.coordinates[0], spb_hex.coordinates[1]), axis=1)
spb_hex

### Add latitude and longitude

In [ ]:
spb_hex['lat'] = spb_hex['coordinates'].apply(lambda x: x[0])
spb_hex['lon'] = spb_hex['coordinates'].apply(lambda x: x[1])
spb_hex

### Getting city name (Takes ling time to compute)

In [ ]:
k = -1
for i in tqdm(spb_hex['coordinates']):
    k += 1
    url = f'https://nominatim.openstreetmap.org/reverse?lat={i[0]}&lon={i[1]}&format=json&addressdetails=1&zoom=5'
    try:
        result = requests.get(url=url)
        result_json = result.json()
        spb_hex.at[k, 'city'] = result_json['address']['state']
    except:
        spb_hex.at[k, 'city'] = None
spb_hex

### Visulazing of hexagons

In [ ]:
def visualize_hexagons(hexagons, color="red", folium_map=None):

    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=20, tiles='cartodbpositron')
    else:
        m = folium_map
        
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m

### Getting the boundaries of Saint Petersburg

In [ ]:
cities = ['Санкт-Петербург']
polygon_krd = ox.geometries_from_place(cities, {'boundary':'administrative'}).reset_index()
polygon_krd = polygon_krd[(polygon_krd['name'] == 'Санкт-Петербург')]
polygon_krd

visualize_polygons(polygon_krd['geometry'])

### Polygon visualization and set the centroid of polygons

In [ ]:
def visualize_polygons(geometry):
    
    lats, lons = get_lat_lon(geometry)
    
    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')
    
    overlay = gpd.GeoSeries(geometry).to_json()
    folium.GeoJson(overlay, name = 'boundary').add_to(m)
    
    return m

# выводим центроиды полигонов
def get_lat_lon(geometry):
        
    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon

### Creating hexagons

In [ ]:
def create_hexagons(geoJson):
    
    polyline = geoJson['coordinates'][0]

    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=9,color="green")
    m.add_child(my_PolyLine)

    hexagons = list(h3.polyfill(geoJson, 9))
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=3,color='red')
        m.add_child(my_PolyLine)
        
    polylines_x = []
    for j in range(len(polylines)):
        a = np.column_stack((np.array(polylines[j])[:,1],np.array(polylines[j])[:,0])).tolist()
        polylines_x.append([(a[i][0], a[i][1]) for i in range(len(a))])
        
    polygons_hex = pd.Series(polylines_x).apply(lambda x: Polygon(x))
        
    return m, polygons_hex, polylines

### Split Saint Petersburg into hexagons

In [ ]:
geoJson = json.loads(gpd.GeoSeries(polygon_krd['geometry']).to_json())
#print(geoJson)
geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson['features'][0]['geometry']['coordinates'][1][0])[:, 1],
                                                      np.array(geoJson['features'][0]['geometry']['coordinates'][1][0])[:, 0])).tolist()]}
m, polygons, polylines = create_hexagons(geoJson)
m

### Work with geojson file from Tele2

In [ ]:
with open('spb_1.geojson', 'r') as j:
    geoJson_tele2_json = json.loads(j.read())
geoJson_tele2_json

In [ ]:
geoJson_tele2 = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson_tele2['features'][0]['geometry']['coordinates'][0])[:, 1],
                                                      np.array(geoJson_tele2['features'][0]['geometry']['coordinates'][0])[:, 0])).tolist()]}
#geoJson_tele2
m, polygons, polylines = create_hexagons(geoJson_tele2)
m

### Save the results

In [ ]:
# для dataframe и object
polygons.to_csv('polygons_tele2.xlsx', index = False)

# для list
file_name = "polylines_tele2.pkl"

open_file = open(file_name, "wb")
pickle.dump(polylines, open_file)
open_file.close()

### The same actions but for larger boundaries (from Tele2)

In [ ]:
# добавил Саня для Tele2
polygon_tele2 = geometry.Polygon(geoJson_tele2_json['features'][0]['geometry']['coordinates'][0])
def osm_query(polygon, tag):
    gdfs_tele2 = ox.features_from_polygon(polygon, tag).reset_index()
    gdfs_tele2['object'] = np.full(len(gdfs_tele2), list(tag.keys())[0])
    gdfs_tele2['type'] = np.full(len(gdfs_tele2), tag[list(tag.keys())[0]])
    gdfs_tele2 = gdfs_tele2[['object', 'type', 'geometry']]
    return gdfs_tele2
  
# Выгрузим интересующие нас категории объектов 
tags = [
        {'building' : 'apartments'}, {'building' : 'detached'}, 
        {'building' : 'dormitory'}, {'building' : 'hotel'}, 
        {'building' : 'house'}, {'building' : 'semidetached_house'}, 
        {'building' : 'terrace'},  {'building' : 'commercial'},
        {'building' : 'office'},  {'building' : 'terrace'},  
        {'building' : 'terrace'}, {'building':'retail'}, 
        {'building':'train_station'}, {'highway' : 'bus_stop'},
        {'amenity':'cafe'}, {'amenity':'fast_food'}, 
        {'amenity':'restaurant'}, {'amenity':'college'}, 
        {'amenity':'language_school'},  {'amenity':'school'},  
        {'amenity':'university'},  
        {'amenity':'bank'},  {'amenity':'clinic'},  
        {'amenity':'hospital'},  {'amenity':'pharmacy'},  
        {'amenity':'theatre'},  {'amenity':'townhall'},
       ]


polygons_tele2 = [polygon_tele2]

gdfs_tele2 = []

for polygon in polygons_tele2:
    for tag in tags:
        gdfs_tele2.append(osm_query(polygon, tag))

In [ ]:
# for list
file_name = "gdfs_tele2.pkl"

open_file = open(file_name, "wb")
pickle.dump(gdfs_tele2, open_file)
open_file.close()

In [ ]:
data_poi_tele2 = pd.concat(gdfs_tele2)
data_poi_tele2.groupby(['object','type'], as_index = False).agg({'geometry':'count'})

lat, lon = get_lat_lon(data_poi_tele2['geometry'])
data_poi_tele2['lat'] = lat
data_poi_tele2['lon'] = lon
data_poi_tele2

### Prepare data to make spatial join of two geo frames

In [ ]:
gdf_1_tele2 = gpd.GeoDataFrame(data_poi_tele2, geometry=gpd.points_from_xy(data_poi_tele2.lon, data_poi_tele2.lat))

gdf_2_tele2 = pd.DataFrame(polygons, columns = ['geometry'])
gdf_2_tele2['polylines'] = polylines
gdf_2_tele2['geometry'] = gdf_2_tele2['geometry'].astype(str)
geometry_uniq = pd.DataFrame(gdf_2_tele2['geometry'].drop_duplicates())
geometry_uniq['id'] = np.arange(len(geometry_uniq)).astype(str)
gdf_2_tele2 = gdf_2_tele2.merge(geometry_uniq, on = 'geometry')
gdf_2_tele2['geometry'] = gdf_2_tele2['geometry'].apply(wkt.loads)
gdf_2_tele2 = gpd.GeoDataFrame(gdf_2_tele2, geometry='geometry')
gdf_2_tele2

In [ ]:
itog_table_tele2 = gpd.sjoin(gdf_2_tele2, gdf_1_tele2)
itog_table_tele2

In [ ]:
itog_table_tele2_temp1 = itog_table_tele2[['geometry', 'object', 'type', 'lat', 'lon', 'id']]
itog_table_tele2_temp1

In [ ]:
itog_table_tele2_temp1.to_excel('objects_osm_merged_with_spb_osm.xlsx', index = False)

In [ ]:
itog_table_tele2_test = gpd.sjoin(gdf_2_tele2, spb_hex_geo)
itog_table_tele2_test

In [ ]:
itog_table_tele2_test.to_excel('hex_tele2_merged_with_spb_osm.xlsx', index = False)

### Merge two dataset in excel

In [ ]:
spb_hex_with_objects = pd.read_excel('spb_hex_with_objects.xlsx', sheet_name='Sheet1', header=0, na_values=['NA'])
spb_hex_with_objects

In [ ]:
spb_hex_with_objects['h3_9'] = spb_hex_with_objects.apply(lambda x:  h3.geo_to_h3(x.lat_spb_hex, x.lon_spb_hex, 9), axis=1)
spb_hex_with_objects

In [ ]:
spb_hex_with_objects.to_excel('spb_hex_with_objects_final.xlsx', index = False)